In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from IPython.display import display, HTML, IFrame
from ipywidgets import interact,fixed, FloatSlider, IntSlider, Dropdown, HBox, VBox
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.patches import Rectangle

from numpy.linalg import norm
from numpy import cos,sin,tan,arctan,exp,log,pi,sqrt,arange,linspace,meshgrid,array

from scipy.integrate import quad, dblquad, tplquad

%matplotlib widget

plt.rcParams.update({
    "figure.figsize": (6, 6),
    "text.usetex": True,
    "font.family": "serif",
})

# Uncomment the one that corresponds to your Jupyter theme
# plt.style.use('default')
plt.style.use('dark_background')
# plt.style.use('fivethirtyeight')
# plt.style.use('Solarize_Light2')

$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\bv}[1]{\begin{bmatrix} #1 \end{bmatrix}}$
$\renewcommand{\vec}{\mathbf}$


In [12]:
%%html

<style>
.shadow {

    /*Edit or add new attributes, change size, color, etc */
    width: 75%;
    box-shadow: 8px 8px 10px #444;
    border: 1px solid silver;

    /*For positioning in a jupyter notebook*/
    margin-top: 2em;
    position: relative;
    top: -25px
}

ol li {
    padding: .5em;
}
</style>

## One-minute Review

<div class="alert alert-block alert-success shadow">
<h3>Follow the gradient, until you can't.</h3>
<p>To find local min/max of a function $f$ on a level set $g = c$, solve 

$$ \nabla f = \lambda \nabla g$$
  
or look where $\nabla g = \vec 0.$</p>
</div>
  
  

  - [Lagrange box/surface area demo](https://drew.youngren.nyc/mvc-f20/lagrange-box.html)
  - [Cobb-Douglas example](https://www.youtube.com/watch?v=QmsYpqTkWqU&t=6s)
  - For help with (numerically) solving systems of equations, see the [Systems of Equations notebook](../extras/SystemsofEquations.ipynb).

<p style="padding-bottom:40%;"> </p>

In [21]:
import sys

# Lecture 14 - Intro to Multiple Integration


  - Objectives

    - Define the double/triple integral

    - Compute using iterated integrals

    - Change the order of integration

  - Resources
    - Content
      - Stewart: §15.1
      - New Strang: [§5.1](https://openstax.org/books/calculus-volume-3/pages/5-1-double-integrals-over-rectangular-regions) [§5.2](https://openstax.org/books/calculus-volume-3/pages/5-2-double-integrals-over-general-regions) [§5.4](https://openstax.org/books/calculus-volume-3/pages/5-4-triple-integrals)
      - [3Demos](https://3demos.surge.sh/?currentChapter=Intro&shadeUp=true&flipInfo=true&grid=false&obj0_kind=graph&obj0_params_a=-2&obj0_params_b=2&obj0_params_c=-2&obj0_params_d=2&obj0_params_z=2+-+%28x+-+2%29%5E2%2F8+-+%28y+-+2%29%5E2%2F16&obj0_params_tau=0&obj0_params_t0=0&obj0_params_t1=1&obj0_params_color=%237d7d7d)
    - Practice
      - Mooculus: [Multiple Integrals](https://ximera.osu.edu/mooculus/calculus3/multipleIntegrals/titlePage) 
    - Extras
      - CalcBLUE: [Integrals](https://www.math.upenn.edu/~ghrist/BLUE.html#VOL3) 

<div style="padding-bottom: 40%"></div>

# Remember Integration?

Recall the definition of a definite integral.

$$\int_a^b f(x)\,dx$$

 $$ = \lim _{N \to \infty} \sum_{i=0}^N f(x_i^*) \Delta x_i$$
 
where $x_i^*$ is a sample point in the $i$th subinterval of $[a,b]$ and $\Delta x_i$ is the width of the $i$th subinterval (often $\frac{b-a}{N}$).

<div style="padding-bottom: 40%"></div>

 $$\lim _{N \to \infty} \sum_{i=1}^N f(x_i^*) \Delta x_i$$


In [13]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))

    plt.ioff()
    fig,ax = plt.subplots(num=TITLE)
    plt.ion()
    
    f = lambda x: 1 - x**2 + x**3/4
    a, b = (0, 2)
    X = np.linspace(-1/2,5/2,100)
    ax.plot(X,f(X),X,np.zeros_like(X),'k');

    slider = IntSlider(min=1, max=44)
    slider2 = FloatSlider(min=0, max=1, step=.05)
    
    N = slider.value
    s = slider2.value
    
    dx = (b-a)/N
    tot = 0
    for i in range(N):
        xi = a + (i)*dx
        yi = f(xi+s*dx)
        tot += yi
        if yi >=0:
            c = 'b'
        else: 
            c = 'r'
        ax.add_patch(Rectangle((xi,0),dx,yi,alpha=.4,color=c))
    ax.set_title(f"$\int_0^2(1-x^2+x^3/4)dx \sim $ {tot * dx:.5f}",fontsize=17)
    
    def update(change):
        N = slider.value
        s = slider2.value
        while ax.patches:
            ax.patches.pop()
        dx = (b-a)/N
        tot = 0
        for i in range(N):
            xi = a + (i)*dx
            yi = f(xi+s*dx)
            tot += yi
            if yi >=0:
                c = 'b'
            else: 
                c = 'r'
            ax.add_patch(Rectangle((xi,0),dx,yi,alpha=.4,color=c))
        ax.set_title(f"$\int_0^2(1-x^2+x^3/4)dx \sim $ {tot * dx:.5f}",fontsize=17)
    
    slider.observe(update, names='value')
    slider2.observe(update, names='value')
    
    return VBox([HBox([slider, slider2]), fig.canvas])

a("2D Riemann")                             

<div style="padding-bottom: 40%"></div>

In [14]:
def f(x): return 1 - x**2 + x**3/4

In [15]:
N = 40000
dx = 2/N
sum([f(0 + (i+1)*dx) for i in range(N)])*dx

0.33328333312500774

<div style="padding-bottom: 40%"></div>

### Extra - Numerical integration in Python

In the real, most integrals are not computed via the Fundamental Theorem of Calculus, but rather _numerically_, i.e., in a process similar to above. The relevant function in the SciPy library is called `quad` (from the old-timey name for integration "quadrature").
 
Note this returns 2 values, the estimated value of the integral, and the estimated error. This one is pretty small. 

In [16]:
from scipy.integrate import quad

quad(f,0,2) # returns tuple (answer, error estimate)

(0.33333333333333326, 1.3047217069399301e-14)

<p style="padding-bottom:40%;"> </p>

# Multiple Integration

Start with a question:

##### What is the average elevation of Bolivia?

<!-- <img alt="Topographical map of Bolivia" src="https://legacy.lib.utexas.edu/maps/americas/bolivia_rel93.jpg" width="50%" />
 -->


In [17]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))
    
    plt.ioff()
    fig, ax = plt.subplots(num=TITLE, figsize=(8,8))
    plt.ion()
    
    im = plt.imread("../img/bolivia_rel93.png")
    
    ax.imshow(im, extent=[0, 536, 0, 622])
    
    slider = IntSlider(min=1, max=20, value=1)
    t = slider.value    
    # ax.grid(True)
    
    def update(change):
        while ax.lines:
            ax.lines.pop()
        t = slider.value
        xs = linspace(0, 536, t + 1)
        ys = arange(0, 623, 536/t)
        for x in xs:
            ax.plot([x, x], [0, 622], 'y')
        for y in ys:
            ax.plot([0, 536], [y, y], 'y')
    
    slider.observe(update, names='value')
    
    return VBox([slider, fig.canvas])

a("Bolivia")



<div style="padding-bottom: 40%"></div>

#### ingredients:
  - **Where?** $\mathcal B$ the set of coordinates $(x,y)$ that make up Bolivia
  - **What?** $f(x,y)$ elevation as a function of horizontal and vertical position
  - **How?** $dA$, the differential, measures small units of area

  $$\text{Average elevation} = f_\text{avg} = \frac{1}{\operatorname{Area}(\mathcal B)} \iint_\mathcal B f\,dA$$

<p style="padding-bottom:40%;"> </p>

# Double Integral

Consider the rectangle $\mathcal{R} = [a,b]\times[c,d] = \{(x,y)\mid a\leq x\leq b, c\leq y \leq d\}$. The **double integral** of a function $f$ over $\mathcal{R}$ is 
$$\iint_\mathcal{R} f(x,y)\,dA $$

$$= \lim_{M,N\to \infty} \sum_{i=1}^M \sum_{j=1}^N f(x_{ij}^*,y_{ij}^*)\, \Delta A_{ij} $$

where $(x_{ij}^*,y_{ij}^*)$ is a sample point in the $ij$th subrectangle of $\mathcal{R}$ and $\Delta A_{ij}$ is the area of the $ij$th subrectangle (often $\Delta x\Delta y$).

A [better integration visualization](https://drew.youngren.nyc/3Demos/iint/) can be found here. 

In [18]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))
    
    plt.ioff()
    fig = plt.figure(num=TITLE)
    plt.ion()
    ax = fig.add_subplot(projection='3d') 
    
    # make axes lines
    ax.plot([-1,1],[0,0],[0,0],'gray')
    ax.plot([0,0],[-1,1],[0,0],'gray')
    ax.plot([0,0],[0,0],[-1,1],'gray')   

    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    
    ax.set_autoscale_on(True)
    
    f = lambda x,y: 1 - x**2 + y**3/10
    a, b, c, d = (0, 2, 1, 4)
    
    X = np.linspace(a,b,150)
    Y = np.linspace(c,d,150)
    X,Y = np.meshgrid(X,Y)

    ax.plot_surface(X,Y,f(X,Y),alpha=.5,cmap='viridis')

    slider = IntSlider(min=3, max = 20)
    slider2 = FloatSlider(min=0, max=1, step=.05)
    
    N, s = slider.value, slider2.value
    
    XN = np.linspace(a,b,N+1)
    YN = np.linspace(c,d,N+1)

    XN,YN = np.meshgrid(XN,YN)
    ZN = (f(XN,YN)).ravel()
    dx = (b-a)/N
    dy = (d-c)/N

    tot = 0
    for i in range(N):
        for j in range(N):
            xi = a + (i)*dx
            yj = c + j*dy
            zi = f(xi+s*dx,yj+s*dy)
            tot += zi
            if zi >=0:
                cl = 'b'
            else: 
                cl = 'r'
            bars = ax.bar3d(xi,yj,0,dx,dy,zi,alpha=.4,color=cl)
#             ax.add_patch(Rectangle((a,c),b-a,d-c))
    ax.set_title(f"$\iint_\mathcal{{R}} (1-x^2+y^3/10) dA \sim $ {tot * dx*dy:.5f}",fontsize=17)
    
    def update(change):
        while len(ax.collections) > 1:
            ax.collections.pop()
        N, s = (slider.value, slider2.value)
    
        XN = np.linspace(a,b,N+1)
        YN = np.linspace(c,d,N+1)

        XN,YN = np.meshgrid(XN,YN)
        ZN = (f(XN,YN)).ravel()
        dx = (b-a)/N
        dy = (d-c)/N

        tot = 0
        for i in range(N):
            for j in range(N):
                xi = a + (i)*dx
                yj = c + j*dy
                zi = f(xi+s*dx,yj+s*dy)
                tot += zi
                if zi >=0:
                    cl = 'b'
                else: 
                    cl = 'r'
                bars = ax.bar3d(xi,yj,0,dx,dy,zi,alpha=.4,color=cl)
    #             ax.add_patch(Rectangle((a,c),b-a,d-c))
        ax.set_title(f"$\iint_\mathcal{{R}} (1-x^2+y^3/10) dA \sim $ {tot * dx*dy:.5f}",fontsize=17)
    
    slider.observe(update, names='value')
    slider2.observe(update, names='value')

    return VBox([HBox([slider, slider2]), fig.canvas])

a("3D Riemann")                
                            

<div style="padding-bottom: 40%"></div>

### Estimate

Approximate the integral of the function $f(x,y)$ whose contour plot is below. Use the idea $$\iint_\mathcal{B} f\, dA = f_\text{avg} \cdot \text{Area}(\mathcal{B})$$

<img src="../img/contour8.png" style="float: left" height="400"> <img src="../img/qr-iintsurvey.png" style="float: right" height="400"> 


<!-- ![Contour Plot of Mystery Function](../img/contour8.png)
 -->

In [19]:
collect_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQRwKlJs72x48sZhqdcvm3NeU92VR-gDhx1MAmwY5WvhDQDD1d1b02_DkvdL4SMbb8T2nWGY0lCVSXt/pub?gid=1709777185&single=true&output=csv"

df = pd.read_csv(collect_url)
entries = df["Eyeball Integration"]

print(df["Eyeball Integration"].describe())
plt.figure(num="Integral estimation")
entries.hist(bins=15)

count     12.000000
mean      41.583333
std       53.239951
min        2.000000
25%        2.750000
50%        7.250000
75%       96.000000
max      128.000000
Name: Eyeball Integration, dtype: float64


<Axes: >

<p style="padding-bottom:40%;"> </p>

<p style="padding-bottom:40%;"> </p>

# Properties of the Double Integral


$$\iint_\mathcal{R} (f(x,y) + g(x,y))\,dA = \iint_\mathcal{R} f(x,y) \,dA + \iint_\mathcal{R} g(x,y)\,dA $$  

$$\iint_\mathcal{R} c(f(x,y)) \,dA = c\iint_\mathcal{R} f(x,y)\,dA$$

$$\iint_\mathcal{R} c \,dA = c\, {\rm Area}(\mathcal{R})$$

<p style="padding-bottom:40%;"> </p>

# Iterated Integrals

#### Quick Question

What sort of mathematical object is this:
$$\int_1^4 f(x,y)\, dy$$

<div style="padding-bottom: 40%"></div>

##### Answer

 It is a function of a single variable $x$. This is sometimes called "integrating out $y$." A student called it a "partial integral." That's cool.

####
<p style="padding-bottom:40%;"> </p>

# Iterated Integrals

An **iterated integral** is a nested collection of integrals like so:

$$ \int_a^b \int_c^d f(x,y)\, dy\, dx = \int_a^b \left( \int_c^d f(x,y)\, dy\right)\, dx $$

In [20]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))
    
    plt.ioff()
    fig = plt.figure(num=TITLE)
    plt.ion()
    ax = fig.add_subplot(projection='3d') 
    
    # make axes lines
    ax.plot([-1,2],[0,0],[0,0],'gray')
    ax.plot([0,0],[-1,4],[0,0],'gray')
    ax.plot([0,0],[0,0],[-2,6],'gray')   

    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    
    ax.set_autoscale_on(True)
    
    f = lambda x,y: 1 - x**2 + y**3/10
    a, b, c, d = (0, 2, 1, 4)
    
    X = np.linspace(a,b,150)
    Y = np.linspace(c,d,150)
    X,Y = np.meshgrid(X,Y)

    ax.plot_surface(X,Y,f(X,Y),alpha=.5,cmap='viridis')

    slider = FloatSlider(min=0, max=1, step=.05)
    drop = Dropdown(options=['x', 'y'], value='x', description='axis')

    
    x = np.linspace(a,b,150)
    y = np.linspace(c,d,150)

    X,Y = np.meshgrid(x,y)
    dx = (b-a)
    dy = (d-c)
    ax.plot_surface(X,Y,f(X,Y),alpha=.5,cmap='viridis')
        
    def update(change):
        while len(ax.collections) > 1:
            ax.collections.pop()
            
        N = drop.value
        s = slider.value

        if N == 'x':
            verts = [(a + s * dx,c,0)]
            tot = 0
            for yi in y:
                zi = f(a + s*dx,yi)
                tot += zi
                verts.append((a + s*dx,yi,zi))
            verts.append((a + s * dx,d,0))
            ax.add_collection3d(Poly3DCollection([verts]))

            ax.set_title(f"$\int_{c}^{d}(1-x^2+y^3/10) dy$ at x={a +s*dx:.2f} is about {tot*(d-c)/150:.5f}",fontsize=15)
        else:
            verts = [(a ,c + s * dy,0)]
            tot = 0
            for xi in x:
                zi = f(xi, c + s*dy)
                tot += zi
                verts.append((xi, c + s*dy, zi))
            verts.append((b, c + s*dy,0))
            ax.add_collection3d(Poly3DCollection([verts]))

            ax.set_title(f"$\int_{a}^{b}(1-x^2+y^3/10) dx$ at y={c +s*dy:.2f} is about {tot*dx/150:.5f}",fontsize=15)
    
    update({'new': 0})
    
    slider.observe(update, names='value')
    drop.observe(update, names='value')

    return VBox([HBox([slider, drop]), fig.canvas])

a("Integrating out")

AttributeError: 'ArtistList' object has no attribute 'pop'

<p style="padding-bottom:40%;"> </p>

## Fubini's Theorem

If $f$ is continuous on the the rectangle $\mathcal{R} = [a,b] \times [c,d]$, then the double integral $\iint_\mathcal{R} f\, dA$ exists and is equal to 

$$ \int_a^b \int_c^d f(x,y)\,dy\,dx =  \int_c^d \int_a^b f(x,y)\,dx\,dy. $$ 

<p style="padding-bottom:40%;"> </p>

### Example

Let $\mathcal{R} = [0,\pi]\times[0,2\pi]$. Evaluate $$\iint_\mathcal{R} x \cos(xy)\,dA$$ as an iterated integral.

<p style="padding-bottom:40%;"> </p>

In [ ]:
(1-cos(2*pi**2))/(2*pi)

In [ ]:
dblquad(lambda y,x: x*cos(x*y),0,pi,0,2*pi)[0]

<p style="padding-bottom:40%;"> </p>

## Nonrectangular domains

Assuming a region $\mathcal{D}$ is not "too bad****", we can integrate over it by admitting to the sum, only terms where the subrectangle lies in $\mathcal{D}$.  

#### Quick Example

Let $\mathcal{D} = \{ (x,y) \mid 1\leq x^2 + y^2 \leq 4\}$, an annulus.

Approximate $$\iint_\mathcal{D} (2+x)\,dA$$

In [ ]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))
    
    plt.ioff()
    fig = plt.figure(num=TITLE)
    plt.ion()
    ax = fig.add_subplot(projection='3d') 
    
    # make axes lines
    ax.plot([-1,1],[0,0],[0,0],'gray')
    ax.plot([0,0],[-1,1],[0,0],'gray')
    ax.plot([0,0],[0,0],[-1,1],'gray')   

    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    
    ax.set_autoscale_on(True)
    
    f = lambda x,y: 2+x 
    a=-2
    b=2
    c=-2
    d=2
    r = np.linspace(1,2,50)
    th = np.linspace(-pi,pi,150)
    # XN = np.linspace(a,b,N+1)
    # YN = np.linspace(c,d,N+1)

    r,th = np.meshgrid(r,th)
    X = r*cos(th)
    Y = r*sin(th)

    ax.plot_surface(X,Y,f(X,Y),alpha=.8,cmap='rainbow')


    slider = IntSlider(min=3, max = 40)
    slider2 = FloatSlider(min=0, max=1, step=.05)
    
    N, s = slider.value, slider2.value
    

    def update(change):
        while len(ax.collections) > 1:
            ax.collections.pop()
        
        N, s = slider.value, slider2.value
            
        # XN,YN = np.meshgrid(XN,YN)
        # ZN = (f(XN,YN)).ravel()
        dx = (b-a)/N
        dy = (d-c)/N
    #     ax.bar3d(XN.ravel(),YN.ravel(),0*ZN,dx,dy,ZN)

        tot = 0
        for i in range(N):
            for j in range(N):
                xi = a + (i)*dx
                yj = c + j*dy
                zi = f(xi+s*dx,yj+s*dy)
                if zi >=0:
                    cl = 'gray'
                else: 
                    cl = 'r'
                if (1 < norm((xi,yj)) < 2) and (1 < norm((xi+dx,yj+dy)) < 2):
                    ax.bar3d(xi,yj,0,dx,dy,zi,alpha=.4,color=cl)
                    tot += zi

        ax.set_title(f"$\int_\mathcal{{D}}(x+2) dA \sim $ {tot * dx*dy:.5f}",fontsize=17)
    
    update({})
    
    slider.observe(update, names='value')
    slider2.observe(update, names='value')

    return VBox([HBox([slider, slider2]), fig.canvas])

a("Nonrectangular Riemann")                
                            

 ****Meaning, essentially, its area is well approximated with rectangles.

<p style="padding-bottom:40%;"> </p>

### Regions defined by functions

A region in $\RR^2$ that is bounded by graphs of functions (of one variable) can be integrated over as interated integrals. 

Suppose $\mathcal{D} = \{(x,y) \mid g(x) \leq y \leq h(x) \}$. Then 

$$\iint_\mathcal{R} f(x,y) \,dA = \int_a^b \int_{g(x)}^{h(x)} f(x,y)\,dy\,dx$$ 

<p style="padding-bottom:40%;"> </p>

## Example

Let $\mathcal{D}$ be the region bounded by the curves $y=x^3/32$ and $y=\sqrt{x}$ in the first quadrant. Write the integral $\iint_\mathcal{D} f(x,y)\,dA$ two ways.

In [ ]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))

    plt.ioff()
    fig,ax = plt.subplots(num=TITLE, figsize=(10,5))
    plt.ion()
    
    x = np.linspace(0,4,100)
    y1 = x**3/32
    y2 = sqrt(x)

    ax.plot(x,y1,x,y2)
    ax.fill_between(x,y1,y2,alpha=.23);
    ax.set_ylim(0,2);
    
    slider = FloatSlider(min=0, max = 4, step=.02)
    drop = Dropdown(options=['x', 'y'], description='axis')
    
    
    def update(change):
        s = slider.value
        axis = drop.value
        
        while ax.patches:
            ax.patches.pop()

        if axis == 'x':
            ax.add_patch(Rectangle([s-.01,s**3/32],.02,sqrt(s)-s**3/32,color='red',alpha=.8))
        else:
            ax.add_patch(Rectangle([(s/2)**2,(s/2)-.01],(32*(s/2))**(1/3)-(s/2)**2,.02,color='red',alpha=.8))
    
    update({})

    slider.observe(update, names='value')
    drop.observe(update, names='value')
    
    return VBox([HBox([slider, drop]), fig.canvas])

a("Any way you slice it")

<p style="padding-bottom:40%;"> </p>

#### Solution

In [ ]:
f = lambda y,x: x*y**3
g = lambda x: x**3/32
h = sqrt

dblquad(f, 0,4,g,h)[0]

In [ ]:
f = lambda x,y: x*y**3
g = lambda y: (32*y)**(1/3)
h = lambda y: y**2

dblquad(f, 0,2,h,g)[0]

#### 
<p style="padding-bottom:40%;"> </p>

### Exercise

We want to find the volume of the solid region under the plane $x-2y+z=10$ and above the region bounded by $x+y=1$ and $x^2+y=1$.

  1. Sketch the region of integration.

  2. Choose an order of integration.

  3. Set up iterated integral.

<p style="padding-bottom:40%;"> </p>

# Triple integrals

Integrating in three dimensions presents no theoretical challenge. 

$$\iiint_E f(x,y,z)\,dV = \lim_{M,N,P\to\infty} \sum_{i=1}^M\sum_{j=1}^N\sum_{k=1}^P f(x_i^*,y_j^*,z_k^*)\,\Delta V_{ijk}$$

### Evaluating

Fubini's Theorem applies directly to this case; thus, we can compute using (3) iterated integrals.

$$\iiint_E f(x,y,z)\,dV = \int_a^b \int_{g(x)}^{h(x)} \int_{j(x,y)}^{k(x,y)} f(x,y,z)\,dz\,dy\,dx $$

<p style="padding-bottom:40%;"> </p>

### Example

Set up an iterated integral to find the volume of the solid region $E$ in the first octant with $$x^2 + y^2 \leq z \leq 4.$$

In [ ]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))
    
    plt.ioff()
    fig = plt.figure(num=TITLE)
    plt.ion()
    ax = fig.add_subplot(projection='3d') 
    
    # make axes lines
    ax.plot([0,2],[0,0],[0,0],'gray')
    ax.plot([0,0],[0,2],[0,0],'gray')
    ax.plot([0,0],[0,0],[0,4],'gray')   

    for c in 'xyz':
        # getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    
    ax.set_autoscale_on(True)
    
    u = np.linspace(0,1,80)
    v = np.linspace(0,1,80)
    U,V = np.meshgrid(u,v)
    X = 2*U*cos(pi/2*V)
    Y = 2*U*sin(pi/2*V)
    Z = 4*U**2
    ax.plot_wireframe(X,Y,Z,rcount=5,ccount=5,color='k')
    ax.plot_surface(X,Y,Z, color='b', alpha=.5)
    ax.plot_wireframe(X,Y,0*Z + 4,rcount=5,ccount=5,color='k')
    ax.plot_surface(X,Y,0*Z + 4, color='b', alpha=.5)    


    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    
    return fig.canvas

a("A Solid Region")

<p style="padding-bottom:40%;"> </p>

In [ ]:
vol = tplquad(lambda z,y,x: 1, 0, 2, 0, lambda x: sqrt(4 - x**2), lambda x,y: x**2 + y**2, 4)[0]
vol

#### Bonus

Suppose a point $(x,y,z)$ is selected randomly from $E$ above. What is the probability that $x > 1$?

<p style="padding-bottom:40%;"> </p>

In [ ]:
event = tplquad(lambda z,y,x: 1, 1, 2, 0, lambda x: sqrt(4 - x**2), lambda x,y: x**2 + y**2, 4)[0]
event/vol*100

In [ ]:
# Let's try it. 

N = 100000
total = 0
event_total = 0
for _ in range(N):
    x,y = 2*np.random.random(2)
    z = 4*np.random.random()
    if x**2 + y**2 <= z:
        total += 1
        if x > 1:
            event_total += 1
print(f"{event_total} out of {total} or {event_total / total * 100}%")

<p style="padding-bottom:40%;"> </p>

## Switching the order of integration

This is not an easy task in 3 variables, but is a very good exercise in spacial reasoning.

It is worth practicing sketching in 3D (and keeping a graphing tool like CalcPlot3D handy). 

### Example

Write the triple integral $\iiint_E f(x,y,z)\,dV$ where E is the region in the first octant bounded by $z = 1-x^2$ and $y = 1-x$ as two different iterated integrals. 

In [ ]:
def a(TITLE):
    if TITLE in plt.get_figlabels():
        plt.close(plt.figure(TITLE))
    
    plt.ioff()
    fig = plt.figure(num=TITLE)
    plt.ion()
    ax = fig.add_subplot(projection='3d') 
    
    # make axes lines
    ax.plot([0,1],[0,0],[0,0],'gray')
    ax.plot([0,0],[0,1],[0,0],'gray')
    ax.plot([0,0],[0,0],[0,1],'gray')   

    for c in 'xyz':
        getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    
    ax.set_autoscale_on(True)
    
    u = np.linspace(0,1,80)
    v = np.linspace(0,1,80)
    U,V = np.meshgrid(u,v)
    X = U
    Y = V*(1-U)
    Z = 1-X**2
    ax.plot_wireframe(X,Y,Z,rcount=5,ccount=5,color='k')
    ax.plot_surface(X,Y,Z, color='b', alpha=.5)
    # ax.plot_surface(X,Y,0*X,rcount=20,ccount=20,color='gray',alpha=.4)
    # ax.plot_surface(U,0*U,(1-U**2)*V,rcount=20,ccount=20,color='gray',alpha=.4)    
    ax.plot_wireframe(U,1-U,V*(1-U**2),rcount=5,ccount=5,color='k')
    ax.plot_surface(U,1-U,V*(1-U**2), color='b', alpha=.5)

    for c in 'xyz':
#         getattr(ax,f"set_{c}lim")([-1,1]);    
        getattr(ax,f"set_{c}label")(f"${c}$",size=16)
    
    return fig.canvas

a("A Triple Integral Example")

<p style="padding-bottom:40%;"> </p>

### Solutions

$$ \int_0^1\int_0^{1-x}\int_0^{1-x^2}f(x,y,z)\,dz\,dy\,dx$$

In [ ]:
tplquad(lambda z,y,x: cos(y)  + sin(z),0,1,0,lambda x: 1 - x,0, lambda x,y: 1- x**2)[0]

$$ \int_0^1\int_0^{1-x^2}\int_0^{1-x}f(x,y,z)\,dy\,dz\,dx$$

In [ ]:
tplquad(lambda y,z,x: cos(y) + sin(z),0,1,0,lambda x: 1 - x**2,0, lambda x,z: 1- x)[0]

#### The hard way

$$ \int_0^1\int_0^{1-(1-y)^2}\int_0^{1-y}f(x,y,z)\,dx\,dz\,dy$$
$$ + \int_0^1\int_{1-(1-y)^2}^1\int_0^{\sqrt{1-z}}f(x,y,z)\,dx\,dz\,dy$$

In [ ]:
tplquad(lambda x,z,y: cos(y) + sin(z),0,1,0,lambda y: 1 - (1-y)**2,0, lambda y,z: 1 - y)[0] + \
tplquad(lambda x,z,y: cos(y) + sin(z),0,1,lambda y: 1 - (1-y)**2,1,0, lambda y,z: sqrt(1-z))[0]